In [1]:
import pandas as pd
import numpy as np
import datetime
from collections import Counter
from py2neo import Graph, Node, Relationship
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
graph = Graph()

In [13]:
def get_similar_films(mov_id):
    query = """
        MATCH (t:Tag)-[:KEYWORD]-(m:Movie)-[:HAS_GENRE]->(g:Genre),
        (t)-[:KEYWORD]-(m2:Movie)-[:HAS_GENRE]->(g)
        WHERE m.movieID = {mov_id} AND m <> m2
        WITH m2, COLLECT(DISTINCT t.name) AS tags, COLLECT(DISTINCT g.name) AS genres
        order by SIZE(genres) DESC, SIZE(tags) DESC
        limit 10
        RETURN collect(distinct m2.movieID) as SimmovieID
        """
    return graph.run(query, mov_id= mov_id).evaluate()

In [44]:
query = """
        MATCH (m:Movie)
        RETURN collect(distinct m.movieID) as movieID
            """
filmlist = list(graph.run(query).evaluate())

In [45]:
cols = ['MovieId', "SimMovieId"]
lst = []
for i in filmlist:
    sim = get_similar_films(i)
    for x in sim:
        lst.append([i, x])

df = pd.DataFrame(lst, columns=cols)


In [46]:
df

,MovieId,SimMovieId
0,147410,40597
1,147410,649
2,147410,133780
3,147657,4438
4,147657,1497
5,147657,46322
6,147657,98499
7,147657,67295
8,147657,6583
9,147657,31878


In [47]:
df.to_csv('../Data/movie_sim.csv', index=False)

In [60]:
import time
import datetime
today = datetime.date.today()
first = today.replace(day=1)
lastMonth = first - datetime.timedelta(days=1)
s= lastMonth.replace(day=1).strftime('%d/%m/%Y')
timestamp = time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y").timetuple())

1561924800.0

In [ ]:
s = "01/7/2019"
